# Object detection

In [10]:
# !mim download mmdet --config mask-rcnn_swin-t-p4-w7_fpn_1x_coco --dest .
%load_ext autoreload
%autoreload 2

from PIL import Image
import torch

from mmdet.apis import init_detector, inference_detector
from mmengine.config import Config

from utils import SplitSwinTransformer, SplitTwoStageDetector, TwoInputIdentity

config_file = 'obj_det/mask-rcnn_swin-t-p4-w7_fpn_1x_coco.py'
checkpoint_file = 'obj_det/mask_rcnn_swin-t-p4-w7_fpn_1x_coco_20210902_120937-9d6b7cfa.pth'

model_init = init_detector(config_file, checkpoint_file, device='cpu')  # or device='cuda:0'
# result = inference_detector(model, 'demo/demo.jpg')

# from mmdet.registry import MODELS
# MODELS.get('SwinTransformer')
# MODELS.get('TwoStageDetector')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loads checkpoint by local backend from path: obj_det/mask_rcnn_swin-t-p4-w7_fpn_1x_coco_20210902_120937-9d6b7cfa.pth


In [13]:
cfg = Config.fromfile('obj_det/mask-rcnn_swin-t-p4-w7_fpn_1x_coco.py')
model = SplitTwoStageDetector.create_from_instance_and_cfg(model_init, cfg, cut_point=1)

prp = model.data_preprocessor
model.data_preprocessor = TwoInputIdentity()

# model.backbone = SplitSwinTransformer.create_from_instance_and_cfg(model.backbone, cfg)

In [20]:
model2 = SplitTwoStageDetector.create_from_cfg_and_checkpoint(config_file, checkpoint_file)



Loads checkpoint by local backend from path: obj_det/mask_rcnn_swin-t-p4-w7_fpn_1x_coco_20210902_120937-9d6b7cfa.pth
[tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), 

In [3]:
import json
from mmengine.registry import RUNNERS
from mmdet.utils import get_test_pipeline_cfg
from utils import get_train_pipeline_cfg

from mmcv.transforms import Compose

cfg = model_init.cfg.copy()
test_pipeline = get_test_pipeline_cfg(cfg)

test_pipeline = Compose(test_pipeline)
train_pipeline = get_train_pipeline_cfg(cfg)

train_pipeline = Compose(train_pipeline)
# data_ = dict(img_path='demo/demo.jpg', img_id=0)


cfg_tr = cfg.copy()
cfg_tr['work_dir'] = './logs'
runner = RUNNERS.build(cfg_tr)
# with open('data/coco/annotations/instances_train2017.json', 'r') as file:
#     data_json = json.load(file)


# data_ = train_pipeline(data_json)

# data_['inputs'] = data_['inputs'].unsqueeze(0)
# data_['data_samples'] = [data_['data_samples']]

# print(train_pipeline)
# print(list(data_json.keys()))

02/23 17:01:56 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.5 (default, May 30 2022, 18:17:32) [GCC 7.5.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1017988511
    GPU 0: NVIDIA GeForce RTX 2080 Ti
    CUDA_HOME: /usr/local/cuda-10.1
    NVCC: Cuda compilation tools, release 10.1, V10.1.24
    GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
    PyTorch: 1.13.1+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_

In [6]:


dl = runner.train_dataloader
print(model.cut_point)
for i,data in enumerate(dl):
    model.zero_grad()
    model.eval()
    # im['inputs'] = torch.stack(im['inputs'])
    # print(list(data_.keys()))
    # data_ = train_pipeline(im)
    # data_ = prp(data_, False)
    # data_['inputs'] = model.backbone.split_forward_v2(data_['inputs'], output_layer=model.cut_point-1)
    # data_['inputs'] = {
    #                "hw_shape": data_['inputs'][0],
    #                "outs": data_['inputs'][1]}
    data = model.feature_frontend(data)
    print(model.backend_loss(data))
    # losses = model.loss(data_['inputs'], data_['data_samples'])
    # loss, losses = model.parse_losses(losses)
    break


1
(tensor(0.3157, grad_fn=<AddBackward0>), OrderedDict([('loss', tensor(0.3157, grad_fn=<AddBackward0>)), ('loss_rpn_cls', tensor(0.0008, grad_fn=<AddBackward0>)), ('loss_rpn_bbox', tensor(0.0214, grad_fn=<AddBackward0>)), ('loss_cls', tensor(0.0573, grad_fn=<MeanBackward0>)), ('acc', tensor(99.0234)), ('loss_bbox', tensor(0.1031, grad_fn=<MeanBackward0>)), ('loss_mask', tensor(0.1332, grad_fn=<MeanBackward0>))]))


In [28]:
# data_ = prp(data_, False)
# data_['inputs'] = model.backbone.split_forward(data_['inputs'], output_layer=model.cut_point-1)
# data_['inputs'] = {"x": data_['inputs'][0],
                #    "hw_shape": data_['inputs'][1],
                #    "outs": data_['inputs'][2]}

# data_['inputs'] = [data_['inputs']]
# data_['data_samples'] = [data_['data_samples']]

# # forward the model
with torch.no_grad():
    results = model.test_step(data_)[0]

AssertionError: 

In [40]:
model.backbone.stages[0]

SwinBlockSequence(
  (blocks): ModuleList(
    (0): SwinBlock(
      (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (attn): ShiftWindowMSA(
        (w_msa): WindowMSA(
          (qkv): Linear(in_features=96, out_features=288, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=96, out_features=96, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
          (softmax): Softmax(dim=-1)
        )
        (drop): DropPath()
      )
      (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (ffn): FFN(
        (layers): Sequential(
          (0): Sequential(
            (0): Linear(in_features=96, out_features=384, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.0, inplace=False)
          )
          (1): Linear(in_features=384, out_features=96, bias=True)
          (2): Dropout(p=0.0, inplace=False)
        )
        (dropout_layer): DropPath()
        (gamma